In [6]:
import pymysql  
import  pymysql.cursors 
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import text 
#https://blog.csdn.net/cugb0451/article/details/45665307 解决百分号报错问题

In [7]:
engine=create_engine('mysql+pymysql://用户名:密码@IP:端口号/库名')
sql = text("SELECT * FROM Inventory limit 8" )
df = pd.read_sql_query(sql, engine)
df.head(3)

,SDate,ShopID,GoodsID,CloseQty,CloseCostV
0,20160101,CDLG,29988848,13.500,14.000
1,20160101,CDLG,29989044,10.684,19.002
2,20160101,CDLG,29989047,15.300,0.040


# 业务背景和价值说明：
项目的实战背景是二家超市的真实数据，数据时间段是从 20160101 到 20171031。
超市的主要商业场景\
1) 对商品的库存管理，是一个零售企业中效率最重要的一环。
2) 商品是否受顾客欢迎，通过商品的动销是一个非常重要的指标。例如：结合库存量具体值，看
看有销售无库存，有库存无销售商品。
3) 商品是否足够有效率，结合商品周转率/天数进行相关分析。【库存周转率是在某一时间段内库存
货物周转的次数。是反映库存周转快慢程度的指标。周转率越大表明销售情况越好。】
商业知识点思考：
1) 结合前面的商品的 ABC 分类与结合商品的库存周转、库存动销是不是就可以对商品有一个相对
完善的分析。
2) 基于这二个版块的指标，是不是可以很好的构建商品的初步画像，及时根据数据对商品进行相
应的上架，下架，退货，促销等的运营动作。
SQL 知识点：
子查询
逻辑说明：
1、 分析计算 8 月的有销售的 SKU 数，库存 SKU 数。
2、 计算期初与期末库存，计算 8 月累计销售。

In [8]:
sql ='''select t3.mmonth
,sku_number
,inven_number
,concat(truncate(sku_number/inven_number*100,2),'%') as rate
from(select month(SDate) as mmonth
,count(distinct GoodsID) as sku_number
from demo.OrderItem t
where t.SDate between '20170801' and '20170831'
group by month(SDate)) t3
join
(select month(t1.SDate) as mmonth
,count(distinct t1.GoodsID) as inven_number
from demo.Inventory t1
where t1.SDate between '20170801' and '20170831'
group by month(t1.SDate)) t4 on t3.mmonth = t4.mmonth;
'''
sql = text(sql)
df = pd.read_sql_query(sql, engine)
df.head(3)

,mmonth,sku_number,inven_number,rate
0,8,10030,10871,92.26%


In [9]:
# 分解
sql_1 = '''
select month(SDate) as mmonth
,count(distinct GoodsID) as sku_number
from demo.OrderItem t
where t.SDate between '20170801' and '20170831'
group by month(SDate)
'''
sql_1 = text(sql_1)
df = pd.read_sql_query(sql_1, engine)
df.head(3)  # 

,mmonth,sku_number
0,8,10030


In [10]:
sql_2='''select month(t1.SDate) as mmonth,count(distinct t1.GoodsID) as inven_number
from demo.Inventory t1 where t1.SDate between '20170801' and '20170831' group by month(t1.SDate);
'''
sql_2 = text(sql_2)
df = pd.read_sql_query(sql_2, engine)
df.head(3)  


,mmonth,inven_number
0,8,10871


In [11]:
sql = '''select t3.mmonth,sku_number
,inven_number
,concat(truncate(sku_number/inven_number*100,1),'%') as rate
from(select month(SDate) as mmonth
,count(distinct GoodsID) as sku_number
from demo.OrderItem t
where t.SDate between '20170801' and '20170831'
group by month(SDate)) t3
join
(select month(t1.SDate) as mmonth
,count(distinct t1.GoodsID) as inven_number
from demo.Inventory t1
where t1.SDate between '20170801' and '20170831'
group by month(t1.SDate)) t4 on t3.mmonth = t4.mmonth;
'''
sql = text(sql)
df = pd.read_sql_query(sql, engine)
df.head(3)

,mmonth,sku_number,inven_number,rate
0,8,10030,10871,92.2%


In [12]:
#---- 库存周转率
#---- 如何需要算周转天数，再用 30 天除以周转率
sql=text('''
select t3.mmonth
,sales_value/avg_inventory_value as zj_rate
from
(select month(SDate) as mmonth
,sum(t.CostValue) as sales_value from 
demo.OrderItem t
where t.SDate between '20170801' and '20170831'
group by month(SDate)) t3
join
(select month(t1.SDate) as mmonth
,sum(t1.CloseCostV)/2 as avg_inventory_value
from demo.Inventory t1
where t1.SDate in ('20170801','20170831')
group by month(t1.SDate)
) t4 
on t3.mmonth = t4.mmonth;
''')
#sql = text(sql)
df = pd.read_sql_query(sql, engine)
df.head()

,mmonth,zj_rate
0,8,11.517451
